In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['font.size'] = 15
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = 16,8

import warnings
warnings.filterwarnings('ignore')


import re
from konlpy.tag import Okt, Mecab
from hanspell import spell_checker
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HOME\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
song_meta = pd.read_json('data/song_meta.json')
song_meta['artist_name'] = song_meta['artist_name_basket'].apply(lambda x : " ".join(x).lower())
song_meta.head()

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,artist_name
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0,various artists
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1,murray perahia
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2,peter gabriel
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3,matoma
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4,jude law


In [4]:
song_meta.query('artist_name == "백현 (baekhyun)"')

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,artist_name
3934,"[GN2502, GN0205, GN2501, GN2506, GN0201]",20190710,City Lights - The 1st Mini Album,10305581,[672859],Psycho (Bonus Track),"[GN2500, GN0200]",[백현 (BAEKHYUN)],3934,백현 (baekhyun)
51681,"[GN2502, GN2501, GN0303, GN2504, GN0301]",20190710,City Lights - The 1st Mini Album,10305581,[672859],Stay Up (Feat. Beenzino),"[GN2500, GN0300]",[백현 (BAEKHYUN)],51681,백현 (baekhyun)
125181,"[GN0401, GN2502, GN2501, GN0402]",20190710,City Lights - The 1st Mini Album,10305581,[672859],Diamond,"[GN0400, GN2500]",[백현 (BAEKHYUN)],125181,백현 (baekhyun)
186196,"[GN2502, GN2501, GN0101, GN2505]",20170414,바래다줄게 (Take You Home) - SM STATION,10055218,[672859],바래다줄게 (Take You Home) (Inst.),"[GN2500, GN0100]",[백현 (BAEKHYUN)],186196,백현 (baekhyun)
312216,"[GN0105, GN1501, GN0101, GN1504]",20200225,낭만닥터 김사부 2 OST,10393897,[672859],너를 사랑하고 있어,"[GN1500, GN0100]",[백현 (BAEKHYUN)],312216,백현 (baekhyun)
486065,"[GN0401, GN2502, GN2501, GN0402]",20190710,City Lights - The 1st Mini Album,10305581,[672859],Ice Queen,"[GN0400, GN2500]",[백현 (BAEKHYUN)],486065,백현 (baekhyun)
505587,"[GN0401, GN2502, GN2501, GN0402]",20190710,City Lights - The 1st Mini Album,10305581,[672859],UN Village,"[GN0400, GN2500]",[백현 (BAEKHYUN)],505587,백현 (baekhyun)
532199,"[GN0105, GN0101, GN1504, GN2502, GN1501, GN250...",20200107,낭만닥터 김사부 2 OST Part.1,10372655,[672859],너를 사랑하고 있어,"[GN2500, GN1500, GN0100]",[백현 (BAEKHYUN)],532199,백현 (baekhyun)
572181,"[GN0105, GN0101]",20151228,2015 가요대전 Limited Edition,2658903,[672859],비처럼 음악처럼,[GN0100],[백현 (BAEKHYUN)],572181,백현 (baekhyun)
607659,"[GN0105, GN0101, GN2505, GN2502, GN1501, GN250...",20200229,하이에나 OST Part.2,10396078,[672859],너에게 가는 이 길 위에서 (너.이.길),"[GN2500, GN1500, GN0100]",[백현 (BAEKHYUN)],607659,백현 (baekhyun)


In [5]:
# 참고--
# 포함 문자열 찾기
temp_str = 'December Holiday Season is finally here!'

for temp in ['Decem', 'mb','day', 'Sea']:
    if temp in temp_str:
        print(temp_str)

December Holiday Season is finally here!
December Holiday Season is finally here!
December Holiday Season is finally here!
December Holiday Season is finally here!


### 해당 가수 찾아오기

In [2]:
from itertools import chain

In [7]:
# 해당 가수 찾아오기
from itertools import chain

def find_singer(names = []):
    
    names = names.split()
    
    idx = []
    for name in names:
        A = [x for x in song_meta['artist_name'] if name in x]
        A = list(set(A))

        
    
        for i in A:
            idx.append(song_meta[song_meta['artist_name'] == i].index.tolist())
    
        idx_list = list(chain.from_iterable(idx))
    
    return song_meta.iloc[idx_list,:]

In [8]:
find_singer('백현')

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,artist_name
180023,"[GN0105, GN0101]",20170214,비가와,10038305,"[490981, 672859]",비가와 (Rain),[GN0100],"[소유 (SOYOU), 백현 (BAEKHYUN)]",180023,소유 (soyou) 백현 (baekhyun)
50974,"[GN2102, GN2101]",20190425,아름다운 초대,10277579,[2640751],아름다운 초대,[GN2100],[백현정],50974,백현정
239509,[GN0701],20070312,The Unique,347021,[221002],그리운 그사람,[GN0700],[백현우],239509,백현우
447007,[GN0201],20090415,2009 프로야구 응원가 (한화이글스),542670,[221002],한화응원가1,[GN0200],[백현우],447007,백현우
21371,[GN0801],20191129,가볍고 수많은,10358776,[168436],빛,[GN0800],[백현진],21371,백현진
...,...,...,...,...,...,...,...,...,...,...
553244,"[GN1502, GN1501]",20021210,철없는 아내와 파란만장한 남편 그리고 태권소녀 OST,25904,"[43467, 168436]",침대 위의 참외,[GN1500],"[정마리, 백현진]",553244,정마리 백현진
466248,"[GN1502, GN1501]",20021210,철없는 아내와 파란만장한 남편 그리고 태권소녀 OST,25904,"[168436, 7343, 686500, 9730]",철없는 아내와 파란만장한 남편 그리고 태권소녀를 위한...,[GN1500],"[백현진, 장영규, 이병훈, 곽경묵]",466248,백현진 장영규 이병훈 곽경묵
352235,"[GN0105, GN0101]",20150921,새시대 통일의 노래 - One Dream One Korea,2641170,"[672859, 247582, 490981, 204065, 602471, 68699...",One Dream One Korea (Original Ver.),[GN0100],"[백현 (BAEKHYUN), 솔지 (EXID), 소유 (SOYOU), 박규리 (카라...",352235,백현 (baekhyun) 솔지 (exid) 소유 (soyou) 박규리 (카라) 핫펠...
249399,"[GN2502, GN0401, GN2501]",20180831,YOUNG,10199959,"[672859, 688432]",YOUNG (Inst.),"[GN0400, GN2500]","[백현 (BAEKHYUN), 로꼬]",249399,백현 (baekhyun) 로꼬


### 해당 제목 찾아오기

In [ ]:
from itertools import chain

def find_title(titles):
    titles = titles.split()
    idx = []
    for title in titles:
        A = [x for x in train['plylst_title'] if title in x]
        A = list(set(A))

        
    
        for i in A:
            idx.append(train[train['plylst_title']  == i].index.tolist())
    
        idx_list = list(chain.from_iterable(idx))
    
    return train.iloc[idx_list,:]

In [ ]:
find_title('9살')

---
---
---

### 사용자 사전 추가

- 가수 이름 추가 
  - 그냥 artist_name으로 나온다 다 정리 할거임
  - 중복 제거할거임
  - 로꼬 / 로꼬 유성은 / 로꼬 화사 이런식으로 저장될거임
  - 이렇게하면 12만개 정도의 이름이 나온다

In [9]:
# 사용자 사전 불러오기

# with open(r"C:\mecab\user-dic\nnp.csv", 'r', encoding='utf-8') as f: 
#     file_data = sorted(f.readlines())
# file_data

In [10]:
# 사용자 사전 추가

# for X in list(set(song_meta['artist_name'].tolist())):
#     file_data.append(f'{X},,,,NNP,*,T,{X},*,*,*,*,*\n')
#     with open("C:/mecab/user-dic/nnp.csv", 'w', encoding='utf-8') as f: 
#         for line in file_data: 
#             f.write(line)

---
---
---

### PyKoSpacing / Py-hanspell

#### pykospacing 다운로드 및 연습

- 내가 볼 때는 이거보다 py-hamspell 성능이 더 좋은 것 같음

In [15]:
# !pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [16]:
from pykospacing import Spacing

In [17]:
sent = '김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.'
new_sent = sent.replace(" ", '') # 띄어쓰기가 없는 문장 임의로 만들기
print(new_sent)

spacing = Spacing()

kospacing_sent = spacing(new_sent) 

print(sent)
print(kospacing_sent)

김철수는극중두인격의사나이이광수역을맡았다.철수는한국유일의태권도전승자를가리는결전의날을앞두고10년간함께훈련한사형인유연재(김광수분)를찾으러속세로내려온인물이다.
김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.
김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.


#### py-hanspell 다운로드 및 연습

In [18]:
# !pip install git+https://github.com/ssut/py-hanspell.git

In [19]:
from hanspell import spell_checker

sent = "맞춤법 틀리면 외 않되? 쓰고싶은대로쓰면돼지 "
spelled_sent = spell_checker.check(sent)

hanspell_sent = spelled_sent.checked
print(hanspell_sent)

맞춤법 틀리면 왜 안돼? 쓰고 싶은 대로 쓰면 되지


In [20]:
spelled_sent = spell_checker.check(new_sent)

hanspell_sent = spelled_sent.checked
print(hanspell_sent)
print(kospacing_sent)

김철수는 극 중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연제(김광수 분)를 찾으러 속세로 내려온 인물이다.
김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.


---
### 자연어처리 

1. 특수기호 제거
2. hanspell로 띄어쓰기 및 맞춤법 처리(붙여쓰기는 불가능)
3. okt 정규화 및 어간 추출 형태소 분리
4. mecab으로 원하는 품사만 추출

In [ ]:
train = pd.read_json('data/train.json')
train['tags__'] = train['tags'].apply(lambda x : " ".join(x))
train['plylst_title__'] = train['plylst_title']
train

In [4]:
stopword = pd.read_csv('data/stopword.csv')
stopword = stopword.iloc[:,0].tolist()

okt = Okt()
m = Mecab(r'C:/mecab/mecab-ko-dic')

In [5]:
def konlpy_preprocessing(text, removes_stopwords = False, stop_words = []):
    # review : 전처리할 텍스트
    # okt : okt 객체를 반복적으로 사용하지 않고 미리 생성한 후 인자로 받는다
    # 불용어 사전은 사용자가 직접 입력해야함. 기본값은 빈 리스트    
    
    
    text = re.sub('[^A-Za-z가-힣0-9]'," ",text)
    txt = spell_checker.check(text).checked
    
    
    # okt를 활용해서 
    # stem = True > 어간 추출
    # norm = True > 정규화 진행
    
    morph = okt.morphs(txt, stem = True, norm = True)

#     sentences_tag = []    
#     sentences_tag.append(morph)
    
    
    part_of_speech = []
    # mecab으로 원하는 품사만 뽑아오기
    for mor in morph:
        for word, tag in m.pos(mor):
            
            # VV 포함해서 다시 돌리기
            if tag in ['NNG', 'NNP', 'NNBC', 'VV','VA', 'XR', 'MAG', 'SN','SL']:
                part_of_speech.append(word.lower())  
            
#             if tag in ['NNG', 'NNP', 'VA', 'XR', 'MAG', 'SL']:
#                 part_of_speech.append(word)
#             elif tag  == 'SN':
#                 b = str(word)
#             elif tag == 'NNBC':
#                 c = str(word)
#                 part_of_speech.append(b+c)                           
    
    part_of_speech = part_of_speech
    
    # stopwords에 있는 단어 제거
    if removes_stopwords == True:
        part_of_speech = [token for token in part_of_speech if not token in stop_words]    

        stops = set(stopwords.words('english')) # 영어 불용어 불러오기

        part_of_speech = [w for w in part_of_speech if not w in stops] 
    
        
    
    return part_of_speech

In [9]:
konlpy_preprocessing('비와이', removes_stopwords = True, stop_words = [])

['비와이']

#### 단위명사

**그냥 1월 ~ 12월, 1살 ~ 10살은 고유명사로 집어넣었음**

---
---
---
### 태그

In [10]:
text = train['tags__'].tolist()
text

['락',
 '추억 회상',
 '까페 잔잔한',
 '연말 눈오는날 캐럴 분위기 따듯한 크리스마스캐럴 겨울노래 크리스마스 겨울왕국 크리스마스송',
 '댄스',
 '운동 드라이브 Pop 트로피컬하우스 힐링 기분전환 2017 팝 트렌드 일렉',
 '짝사랑 취향저격 슬픔 고백 사랑 이별',
 '잔잔한 추억 회상',
 '일렉트로니카 포크 메탈 락 댄스 인디',
 '록 Metal 이일우 M에센셜 메탈 Rock 락',
 'kpop 댄스 걸그룹댄스 스트레스해소',
 '새해 여행 프로필음악 카카오톡 기분전환 소원 프로필 소망 다짐 카톡',
 '듣고 우울 힐링 이거 힘내',
 '힙합 느낌있는 밤 새벽 RnB 감각적인 드라이브 국내 그루브한',
 '가을 재즈',
 '락',
 '감성 질리지않는 나만알고싶은 Pop',
 '봄 설렘 사랑',
 '비오는날 누군가생각날때',
 '스밍 목록 폐막식 올림픽 엑소',
 '조용히 혼자 또는 새벽감성 고민 맥주한잔',
 '카페 재즈 잔잔한',
 'OST',
 '댄스',
 '감성 어쿠스틱 잔잔한 새벽 편안한 인디 밤',
 '잔잔한 밤 새벽',
 '힐링 드라이브 에너지 인디 여행',
 '듀엣 취향저격 달달 피쳐링 사랑 남녀',
 '까페 설렘 사랑',
 '힙합 신인 국힙 인디 신나는',
 '거슈윈 존윌리엄스 랩소디인블루 미국클래식 영화클래식 아메리칸클래식',
 '겨울 기분전환',
 '아침 빅밴드 상쾌한 재즈 밝은 스윙 스윙재즈',
 '휴식 힐링 설렘 사랑',
 '여름 일렉트로니카 매장음악 취향저격 댄스 드라이브 여행 기분전환 일렉 신나는',
 '까페 설렘 사랑',
 '잔잔한 설렘 사랑',
 '헬스 스포츠 피트니스 운동 다이어트 런닝 레깅스 필라테스 산책 요가',
 '발라드',
 '겨울 크리스마스 캐롤',
 '위로 힐링',
 '기분전환 가을',
 '80년대 감성 90년대발라드 올드송 추억 휴식 90년대 옛날노래',
 '월드 연주 재즈',
 '잔잔한 밤 새벽',
 '슬픔 밤 이별 새벽',
 '클럽 스트레스',
 '휴식 주말 오후 나른한 Chillout 편안

In [12]:
tags = []
tag1 = [konlpy_preprocessing(txt, removes_stopwords=True, stop_words=stopword) for txt in text[:20001]]
tags.append(tag1)
display(len(tag1))
display(len(tags))

20001

1

In [13]:
tag2 = [konlpy_preprocessing(txt, removes_stopwords=True, stop_words=stopword) for txt in text[20001:40001]]
tags.append(tag2)
display(len(tag2))
display(len(tags))

20000

2

In [14]:
tag3 = [konlpy_preprocessing(txt, removes_stopwords=True, stop_words=stopword) for txt in text[40001:60001]]
tags.append(tag3)
display(len(tag3))
display(len(tags))

20000

3

In [15]:
tag4 = [konlpy_preprocessing(txt, removes_stopwords=True, stop_words=stopword) for txt in text[60001:80001]]
tags.append(tag4)
display(len(tag4))
display(len(tags))

20000

4

In [16]:
tag5 = [konlpy_preprocessing(txt, removes_stopwords=True, stop_words=stopword) for txt in text[80001:100001]]
tags.append(tag5)
display(len(tag5))
display(len(tags))

20000

5

In [18]:
tag6 = [konlpy_preprocessing(txt, removes_stopwords=True, stop_words=stopword) for txt in text[100001:]]
tags.append(tag6)
display(len(tag6))
display(len(tags))

15070

6

In [21]:
tag = list(chain.from_iterable(tags))
tag

[['락'],
 ['추억', '회상'],
 ['카페', '잔잔'],
 ['연말',
  '눈',
  '오다',
  '날',
  '캐럴',
  '분위기',
  '따',
  '크리스마스',
  '캐럴',
  '겨울',
  '노래',
  '크리스마스',
  '겨울',
  '왕국',
  '크리스마스',
  '송'],
 ['댄스'],
 ['운동',
  '드라이브',
  'pop',
  '트로피',
  '컬',
  '하우스',
  '힐링',
  '기분전환',
  '2017',
  '팝',
  '트렌드',
  '일렉'],
 ['짝사랑', '취향', '저격', '슬픔', '고백', '사랑', '이별'],
 ['잔잔', '추억', '회상'],
 ['일렉트로니카', '포크', '메탈', '락', '댄스'],
 ['록', 'metal', '이일우', '에센셜', '메탈', 'rock', '락'],
 ['k', 'pop', '댄스', '그룹', '댄스', '스트레스', '해소'],
 ['새해',
  '여행',
  '프로필',
  '음악',
  '카카오',
  '톡',
  '기분전환',
  '소원',
  '프로필',
  '소망',
  '다',
  '짐',
  '카톡'],
 ['듣', '우울', '힐링', '힘내'],
 ['힙합', '느낌', '밤', '새벽', 'rnb', '감각', '드라이브', '국내', '그루브'],
 ['가을', '재즈'],
 ['락'],
 ['감성', '질리', 'pop'],
 ['봄', '설레', '사랑'],
 ['비', '오다', '날', '생각나'],
 ['스미다', '목록', '폐막식', '올림픽', '엑소'],
 ['조용', '새벽', '감성', '고민', '맥주', '잔'],
 ['카페', '재즈', '잔잔'],
 ['ost'],
 ['댄스'],
 ['감성', '어쿠스틱', '잔잔', '새벽', '편안', '밤'],
 ['잔잔', '밤', '새벽'],
 ['힐링', '드라이브', '에너지', '여행'],
 ['듀엣', '취향', '저격', '달달',

In [26]:
tag_df = pd.DataFrame(np.concatenate(tags))
tag_df.columns = ['tags__']
tag_df

,tags__
0,[락]
1,"[추억, 회상]"
2,"[카페, 잔잔]"
3,"[연말, 눈, 오다, 날, 캐럴, 분위기, 따, 크리스마스, 캐럴, 겨울, 노래, ..."
4,[댄스]
...,...
115066,"[록, 메탈, 밴드, 사운드, 록, 락, 메탈, 메탈, 락, extreme]"
115067,[일렉]
115068,"[담시, 가족, 눈물, 그리움, 주인공, 이야기, 사랑, 친구]"
115069,"[잔잔, 버스, 퇴근, 버스, pop, 풍경, 퇴근길]"


---
### 타이틀을 태그로

In [6]:
test = train['plylst_title__'].tolist()
test

['여행같은 음악',
 '요즘 너 말야',
 '편하게, 잔잔하게 들을 수 있는 곡.-',
 '크리스마스 분위기에 흠뻑 취하고 싶을때',
 '추억의 노래 ㅋ',
 '2017 Pop Trend',
 '짝사랑..고백..사랑..이별..슬픔.. 감성을 자극하는곡들!',
 '멍청이.. 내맘도 몰라.',
 'DANCING IN THE MOON-LIGHT .01',
 '[록/메탈] Written by 이일우',
 '걸그룹 땐쓰쏭 ',
 '노래로 의지를 불태우자! "1일1다짐" st용 프로필뮤직',
 '지친 너를 위로해줄 제목편지 준비해봤어',
 '트렌디하고 그루브한 힙합/알앤비 MUSIC',
 'Autumn in Jazz',
 '걍게임할때듣는음악',
 '나만 알고싶은 노래들',
 '축가듀엣',
 '이렇게 비 내리는 날이면, 너도 내 생각 할까..',
 '올림픽 스밍 목록',
 '조용히 맥주 한잔 할때_새벽감성',
 'ᴡʜɪᴛᴇ : ʀᴏᴍᴀɴᴛɪᴄ ᴊᴀᴢᴢ ᴀᴛ ᴀ ᴄᴀғᴇ',
 '추억의 명화를 만나는 시간 (영화 OST 모음)',
 '★☆기분좋은 여름날☆★',
 '지친 하루 끝, 힐링이 필요한 당신에게 추천하는 인디곡',
 '새벽 찬바람이 침대위를 감쌀때..(국외)',
 '기분좋은 햇살같은 숨겨진 명곡들 ❤',
 '사랑스러운 남녀듀엣',
 '즐겨듣는소래',
 '닥힙추 8월 Playlist',
 '거슈윈에서 스타워즈까지, 미국 대표 클래식',
 '겨울이 기다리며 미리 들어요^^~',
 '아침이 상쾌해지는 빅밴드의 스윙스윙 ♬',
 '여행가는 길 이런음악과 함께 고고씽!!!',
 '믿고듣는 하우스 일렉트로니카 POP #2',
 '프로포즈 음악',
 '보아일본곡',
 '안다르 레깅스 입고 운동 시작하자!',
 '발라드로 감성을 발라드림(슬픔주의)',
 '크리스마스를 책임질 캐롤 모음',
 '아무것도 위로 되지 못할 때 위로가 되어준 음악',
 '트렌디한 Cafe Pop Music',
 '8~90년대 우리나라 라디오에서 흘러나오던 명곡들.',
 '영화, Jazz를 만나다 294',

In [14]:
len(title_to_tags) 

4

In [8]:
title_to_tags = []
title_to_tag1 = [konlpy_preprocessing(txt,removes_stopwords=True, stop_words=stopword) for txt in test[:20001]]
title_to_tags.append(title_to_tag1)
display(len(title_to_tag1))
display(len(title_to_tags))

20001

1

In [9]:
title_to_tag2 = [konlpy_preprocessing(txt,removes_stopwords=True, stop_words=stopword) for txt in test[20001:40001]]
title_to_tags.append(title_to_tag2)
display(len(title_to_tag2))
display(len(title_to_tags))

20000

2

In [10]:
title_to_tag3 = [konlpy_preprocessing(txt,removes_stopwords=True, stop_words=stopword) for txt in test[40001:60001]]
title_to_tags.append(title_to_tag3)
display(len(title_to_tag3))
display(len(title_to_tags))

20000

3

In [11]:
title_to_tag4 = [konlpy_preprocessing(txt,removes_stopwords=True, stop_words=stopword) for txt in test[60001:80001]]
title_to_tags.append(title_to_tag4)
display(len(title_to_tag4))
display(len(title_to_tags))

20000

4

In [15]:
title_to_tag5 = [konlpy_preprocessing(txt,removes_stopwords=True, stop_words=stopword) for txt in test[80001:100001]]
title_to_tags.append(title_to_tag5)
display(len(title_to_tag5))
display(len(title_to_tags))

20000

5

In [16]:
title_to_tag6 = [konlpy_preprocessing(txt,removes_stopwords=True, stop_words=stopword) for txt in test[100001:]]
title_to_tags.append(title_to_tag6)
display(len(title_to_tag6))
display(len(title_to_tags))

15070

6

In [17]:
title_to_tag = list(chain.from_iterable(title_to_tags))
title_to_tag

[['여행', '음악'],
 ['요즘'],
 ['편하', '잔잔', '곡'],
 ['크리스마스', '분위기', '흠뻑', '취하'],
 ['추억', '노래'],
 ['2017', 'pop', 'trend'],
 ['짝사랑', '고백', '사랑', '이별', '슬픔', '감성', '자극', '곡'],
 ['멍청이', '맘', '도'],
 ['dancing', 'moon', 'light', '01'],
 ['록', '메탈', 'written', '이일우'],
 ['그룹', '쓰', '쏭'],
 ['노래', '의지', '불', '태우', '1', '일', '1', '다', '짐', 'st', '용', '프로필', '뮤직'],
 ['지치', '위로', '제목', '편지', '준비'],
 ['트렌디', '그루브', '힙합', '알앤비', 'music'],
 ['autumn', 'jazz'],
 ['그냥', '게임', '듣', '음악'],
 ['노래'],
 ['축가', '듀엣'],
 ['비', '내리', '날', '도'],
 ['올림픽', '스미다', '목록'],
 ['조용', '맥주', '한잔', '새벽', '감성'],
 [],
 ['추억', '명화', '만나', '영화', 'ost', '모음'],
 ['기분', '여름날'],
 ['지치', '하루', '끝', '힐링', '필요', '추천', '인기', '곡'],
 ['새벽', '바람', '침대', '위', '감싸', '국외'],
 ['기분', '햇살', '숨기', '명곡'],
 ['사랑', '남녀', '듀엣'],
 ['즐기', '듣', '소래'],
 ['닥힙추', '8월', 'playlist'],
 ['거슈윈', '스타워즈', '미국', '대표', '클래식'],
 ['겨울', '기다리', '미리'],
 ['아침', '상쾌', '빅밴드', '스윙', '스윙'],
 ['여행', '길', '음악', '고고씽'],
 ['믿', '듣', '하우스', '일렉트로니카', 'pop', '2'],
 ['프러포즈', '음악'],
 ['

In [18]:
title_to_tag_df = pd.DataFrame(np.concatenate(title_to_tags),columns=['plylist_title__'])
title_to_tag_df

,plylist_title__
0,"[여행, 음악]"
1,[요즘]
2,"[편하, 잔잔, 곡]"
3,"[크리스마스, 분위기, 흠뻑, 취하]"
4,"[추억, 노래]"
...,...
115066,"[metal, e, sm, 2]"
115067,"[빠르, 리스, 위, 따끈따끈, 최신, 인기, edm, 모음]"
115068,"[1, 눈물, 가리, 이야기]"
115069,"[퇴근, 버스, 편하, 하루, 마무리, pop]"


In [42]:
tag_df.to_json('data/tag_konlpy.json', orient = 'records')
title_to_tag_df.to_json('data/title_to_tag_konlpy.json', orient = 'records')